In [38]:
from LogAnalyzer import logAnalyzer
import pandas as pd
import re

In [39]:
la = logAnalyzer(ip_list=[], log_dir="./logs/")

# la.get_logs()
la.read_logs()

logs = la.logs_df

  0%|          | 0/5 [00:00<?, ?it/s]c:\Users\phili\Documents\PyProjects\gethLogAnalyzer\LogAnalyzer.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(df, ignore_index=True)
c:\Users\phili\Documents\PyProjects\gethLogAnalyzer\LogAnalyzer.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(df, ignore_index=True)
100%|██████████| 5/5 [00:00<00:00, 29.07it/s]


In [40]:
logs["timestamp"] = pd.to_datetime(logs["timestamp"], format='[%m-%d|%H:%M:%S.%f]') + pd.DateOffset(years=122, hours=1)

In [41]:
logs.dropna(inplace=True)
logs=logs[logs["message"].str.contains("Imported new chain segment")]


In [42]:
logs["message"] = logs["message"].map(lambda x: re.sub(' +', ' ', x.lstrip("Imported new chain segment")))

C:\Users\phili\AppData\Local\Temp\ipykernel_11216\1340585868.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  logs["message"] = logs["message"].map(lambda x: re.sub(' +', ' ', x.lstrip("Imported new chain segment")))


In [43]:
"blocks=1   txs=41   mgas=1.005  elapsed=49.407ms    mgasps=20.347  number=128 hash=761d67..4e8cf5 age=1m20s    dirty=100.21KiB"
logs[["blocks", "txs", "mgas", "elapsed", "mgasps", "number", "hash", "age", "dirty"]] = logs["message"].str.split(" ", expand=True)
logs.drop(columns=['level', 'message', 'age', 'dirty'], inplace=True)

C:\Users\phili\AppData\Local\Temp\ipykernel_11216\2170069253.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  logs[["blocks", "txs", "mgas", "elapsed", "mgasps", "number", "hash", "age", "dirty"]] = logs["message"].str.split(" ", expand=True)
C:\Users\phili\AppData\Local\Temp\ipykernel_11216\2170069253.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  logs[["blocks", "txs", "mgas", "elapsed", "mgasps", "number", "hash", "age", "dirty"]] = logs["message"].str.split(" ", expand=True)
C:\Users\phili\AppDa

In [44]:
logs[["blocks", "txs", "mgas", "elapsed", "mgasps", "number", "hash"]]=logs[["blocks", "txs", "mgas", "elapsed", "mgasps", "number", "hash"]].apply(lambda x: x.str.split("=").str[1])
logs = logs.drop_duplicates(subset='hash', keep="first")
logs.sort_values(by="timestamp", inplace=True)


C:\Users\phili\AppData\Local\Temp\ipykernel_11216\701800437.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  logs[["blocks", "txs", "mgas", "elapsed", "mgasps", "number", "hash"]]=logs[["blocks", "txs", "mgas", "elapsed", "mgasps", "number", "hash"]].apply(lambda x: x.str.split("=").str[1])


In [45]:
def filter_and_export(df, start, end, dt_col="timestamp", outfile="outfile.csv"):
    filt_df = df[(df[dt_col] >= start) & (df[dt_col] <= end)]
    if outfile:
        filt_df.to_csv(outfile)
        print(f"Exported to {outfile}")
    return filt_df

In [46]:
filter_and_export(logs, "2022-12-18 21:30:00", "2022-12-18 22:30:00", outfile="1strun.csv")

Exported to 1strun.csv


,timestamp,filename,blocks,txs,mgas,elapsed,mgasps,number,hash
147,2022-12-18 21:30:28.324,2022_12_18_08_24_pm_19216804.log,1,62,1.523,43.821ms,34.746,121,77edac..38cf3a
194,2022-12-18 21:32:29.484,2022_12_18_08_24_pm_19216804.log,1,61,1.499,43.593ms,34.381,122,e3da84..a9c02b
25321,2022-12-18 21:32:40.508,2022_12_18_08_24_pm_19216805.log,1,5,0.122,32.455ms,3.758,123,43b9a2..664404
25333,2022-12-18 21:33:06.206,2022_12_18_08_24_pm_19216805.log,1,13,0.321,18.357ms,17.490,124,02b810..e37b8b
229,2022-12-18 21:33:36.248,2022_12_18_08_24_pm_19216804.log,1,15,0.363,19.725ms,18.410,125,4683bb..704188
...,...,...,...,...,...,...,...,...,...
1665,2022-12-18 22:26:14.379,2022_12_18_08_24_pm_19216804.log,1,15,0.369,21.226ms,17.372,208,2fcd0c..962f1e
1668,2022-12-18 22:26:15.672,2022_12_18_08_24_pm_19216804.log,1,1,0.027,13.162ms,2.024,209,6a43e8..0a1156
1672,2022-12-18 22:26:20.602,2022_12_18_08_24_pm_19216804.log,1,2,0.048,13.169ms,3.620,210,e14501..7f7c92
1699,2022-12-18 22:27:29.428,2022_12_18_08_24_pm_19216804.log,1,35,0.865,30.996ms,27.909,211,eca486..040274
